# Scraping Top 50 movies from IMDb

In [1]:
#Importing libraries

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#Connect to a website
#Assiging URL variable name as url

url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating'

#request allow you to send HTTP requests
response = requests.get(url)           
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
#Creating empty lists to append values

movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
description = []
Director = []
Stars = []

In [5]:
#Storing the required data in variable

movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

In [6]:
#Calling one by one variables using for loop

for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)
    
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
    
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)
    #since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    vote = value[0].text
    votes.append(vote)
    
    grosses = value[1].text if len(value) >1 else '*****'
    gross.append(grosses)
    
    # Description of the Movies  
    describe = store.find_all('p', class_ = 'text-muted')
    description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
    description.append(description_)
    
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    Director.append(cast[0])
    Stars.append([x.strip() for x in cast[1].split(",")])

In [8]:
movie_DF = pd.DataFrame({'Name of Movie' : movie_name, 'Year of release' : year, 'Watchtime' : time, 'Movie Rating' : rating, 'Metascore' : metascore, 'Votes' : votes, 'Gross Collection' : gross, 'Description' : description, 'director' : Director, 'star' : Stars})

In [12]:
movie_DF.to_csv("Top_50_IMDb_Movies.csv")

In [13]:
movie_DF.head()

,Name of Movie,Year of release,Watchtime,Movie Rating,Metascore,Votes,Gross Collection,Description,director,star
0,The Shawshank Redemption,1994,142,9.3,82,"2,778,346",$28.34M,"Over the course of several years, two convicts...",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,934,327",$134.97M,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,2008,152,9.0,84,"2,755,411",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,Schindler's List,1993,195,9.0,95,"1,398,311",$96.90M,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."
4,12 Angry Men,1957,96,9.0,97,"824,577",$4.36M,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
